<a href="https://colab.research.google.com/github/tmskss/ManPageSum/blob/main/colab/finetune_falcon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!git clone https://github.com/tmskss/ManPageSum.git
%cd ManPageSum/colab
from install import *
install_requirements()

from utils import *
setup()

Cloning into 'ManPageSum'...
remote: Enumerating objects: 1234, done.
remote: Counting objects: 100% (1234/1234), done.
remote: Compressing objects: 100% (950/950), done.
remote: Total 1234 (delta 307), reused 1199 (delta 283), pack-reused 0
Receiving objects: 100% (1234/1234), 9.24 MiB | 18.27 MiB/s, done.
Resolving deltas: 100% (307/307), done.
/content/ManPageSum/colab/ManPageSum/colab/ManPageSum/colab/ManPageSum/colab
⏳ Installing base requirements ...
✅ Base requirements installed!
✅ Summary requirements installed!


TypeError: ignored

In [ ]:
!pip install ludwig
!pip install ludwig[llm]

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

def clear_cache():
  if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [ ]:
import getpass
import locale; locale.getpreferredencoding = lambda: "UTF-8"
import logging
import os
import torch
import yaml

from ludwig.api import LudwigModel


os.environ["HUGGING_FACE_HUB_TOKEN"] = getpass.getpass("Token:")
assert os.environ["HUGGING_FACE_HUB_TOKEN"]

In [ ]:
from datasets import load_dataset

dataset = load_dataset("tmskss/linux-man-pages-tldr-summarized")
print(f"Features: {dataset['train'].column_names}")

In [ ]:
import random

def create_validation_split(dataset, validation_percent=0.1):

    # Calculate the number of rows for the validation set
    num_validation_samples = int(len(dataset['train']) * validation_percent)

    # Randomly shuffle the dataset
    shuffled_indices = list(range(len(dataset['train'])))
    random.shuffle(shuffled_indices)

    # Select rows for the validation set
    validation_indices = shuffled_indices[:num_validation_samples]
    validation_data = [dataset['train'][i] for i in validation_indices]

    # Remove validation rows from the original dataset
    training_data = [dataset['train'][i] for i in shuffled_indices[num_validation_samples:]]

    dataset['validation'] = validation_data

    return dataset

In [ ]:
dataset = create_validation_split(dataset)

print('The dataset has {} rows of training data and {} rows of validation data'.format(len(dataset['train']), len(dataset['validation'])))